# Inventories

Inventories are a fundamental doc entrypoint for our infrastructures. 

They contain a lot of informations, including:
    
    - ansible_user
    - configuration variables
    - host grouping eg. by geographical zones
    
    

In [5]:
cd exercise-04

/notebooks/exercise-04


In [6]:
!cat inventory

# I can group hosts in inventory

[web:children]
web_rome
web_milan

[web_rome:children]
web_rome_test
web_rome_prod

# further host variables
[web_rome:vars]
ansible_username=root 

# Connect with docker 
[web_rome_test:vars]
ansible_connection=docker
ansible_docker_extra_args="-Htcp://172.17.0.1"



# The actual host reference
[web_rome_test]
ansible101_web_1


[web_rome_prod]
172.23.0.[3:4]

[web_milan]
172.24.0.[5:6]


In [7]:
# The ping module is very useful. Use it whenever you want to check connectivity!
!ansible -m ping -i inventory all


ansible101_web_1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.23.0.3 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.23.0.3 port 22: Network is unreachable\r\n", 
    "unreachable": true
}
172.23.0.4 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.23.0.4 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.24.0.5 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.24.0.5 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.24.0.6 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.24.0.6 port 22: Connection timed out\r\n", 
    "unreachable": true
}


## The setup module

It's very useful. When running ansible playbooks you can access all the variables it gathers

In [15]:
# The After the ping module, there's another one which is very useful
# Exercise:
# - run the following command
# - what does it do?
# - where's  its output ?
# - use the json module to reindent it
!ansible >/dev/null -i inventory -m setup ansible101_web_1 --tree host_status

In [28]:
# Hide the following in some way XXX
ret = !cat host_status/ansible101_web_1
import json
json.loads(ret.s)

{u'ansible_facts': {u'ansible_all_ipv4_addresses': [u'172.17.0.3'],
  u'ansible_all_ipv6_addresses': [u'fe80::42:acff:fe11:3'],
  u'ansible_apparmor': {u'status': u'disabled'},
  u'ansible_architecture': u'x86_64',
  u'ansible_bios_date': u'06/30/2014',
  u'ansible_bios_version': u'A00',
  u'ansible_cmdline': {u'BOOT_IMAGE': u'/vmlinuz-4.10.17-100.fc24.x86_64',
   u'LANG': u'it_IT.UTF-8',
   u'i8042.nopnp': True,
   u'rd.lvm.lv': u'vg0/root00',
   u'ro': True,
   u'root': u'/dev/mapper/vg0-root00'},
  u'ansible_date_time': {u'date': u'2017-06-26',
   u'day': u'26',
   u'epoch': u'1498503105',
   u'hour': u'18',
   u'iso8601': u'2017-06-26T18:51:45Z',
   u'iso8601_basic': u'20170626T185145691667',
   u'iso8601_basic_short': u'20170626T185145',
   u'iso8601_micro': u'2017-06-26T18:51:45.691728Z',
   u'minute': u'51',
   u'month': u'06',
   u'second': u'45',
   u'time': u'18:51:45',
   u'tz': u'UTC',
   u'tz_offset': u'+0000',
   u'weekday': u'Monday',
   u'weekday_number': u'1',
   u'wee

In [27]:
# Exercise: gather the following infos from the ansible setup module
# - hostname
# - first ip address
# - distribution
# - kernel release


## Inventory scripts

In [22]:
#To create custom inventory scripts just use python ;) and set it in

!grep inventory  ansible.cfg #inventory = ./docker-inventory.py


inventory = inventory-docker.py


In [11]:
# Exercise: in the official ansible documentation find at least 3 `ansible_connection=docker` parameters 

In [26]:
# List our containers. Note: this only works with docker-compose containers.
from __future__ import print_function
import docker
c=docker.Client(base_url="http://172.17.0.1:2375")
container_fmt = lambda x: (
    x['Names'][0][1:],
    x['Labels']['com.docker.compose.service'], 
    x['NetworkSettings']['Networks']['bridge']['IPAddress'],
)

for x in c.containers():
    print(*container_fmt(x), sep='\t\t')

ansible101_web_6		web		172.17.0.9
ansible101_web_3		web		172.17.0.8
ansible101_web_4		web		172.17.0.7
ansible101_web_5		web		172.17.0.6
ansible101_ansible_1		ansible		172.17.0.5
ansible101_web_2		web		172.17.0.4
ansible101_web_1		web		172.17.0.3
ansible101_dev_1		dev		172.17.0.2


In [34]:
# Ansible accepts
import json

inventories = {
    'web': {
        'hosts': ['ws-1', 'ws-2'],
    },
    'db': {
        'hosts': ['db-1', 'db-2'],
    }
}

# like this 
print(json.dumps(inventories, indent=1))
      

{
 "web": {
  "hosts": [
   "ws-1", 
   "ws-2"
  ]
 }, 
 "db": {
  "hosts": [
   "db-1", 
   "db-2"
  ]
 }
}


In [35]:
# You can pass variables to generated inventories too
inventories['web']['host_vars'] = {
    'ansible_ssh_common_args': ' -o GSSApiAuthentication=no'
}

print(json.dumps(inventories, indent=1))


{
 "web": {
  "hosts": [
   "ws-1", 
   "ws-2"
  ], 
  "host_vars": {
   "ansible_ssh_common_args": " -o GSSApiAuthentication=no"
  }
 }, 
 "db": {
  "hosts": [
   "db-1", 
   "db-2"
  ]
 }
}


In [40]:
# Exercise: using the above code, create a custom inventory-docker.py that
# - connects via docker to "web" hosts
# - connects via ssh to "ansible" hosts 



In [12]:
# Test our custom inventory-docker.py file
!ansible -m ping -i inventory-docker.py all --limit ansible


ERROR! Attempted to execute "inventory-docker.py" as inventory script: Inventory script (inventory-docker.py) had an execution error: DEBUG:docker.auth.auth:Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
DEBUG:docker.auth.auth:No config file found
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 172.17.0.1
DEBUG:urllib3.connectionpool:http://172.17.0.1:2375 "GET /v1.24/containers/json?all=0&limit=-1&trunc_cmd=0&size=0 HTTP/1.1" 200 None
DEBUG:root:Processing entry u'ansible101_dev_1\t\tdev\t\t172.17.0.2'
DEBUG:root:Processing entry u'ansible101_ansible_1\t\tansible\t\t172.17.0.5'
DEBUG:root:Processing entry u'ansible101_web_2\t\tweb\t\t172.17.0.4'
DEBUG:root:Processing entry u'ansible101_web_1\t\tweb\t\t172.17.0.3'
Traceback (most recent call last):
  File "/notebooks/exercise-04/inventory-docker.py", line 25, in <module>
    inventory['web']['ansible_ssh_common_args'] = ' -o StrictHostKeyChecking=no '
TypeError: list indices must be integers, not str
 


In [ ]:
# Now modify the inventory-docker.py to skip StrictHostKeyChecking only on web hosts.

# Configurations

You may want to split inventory files and separate prod and test environment.

Exercise: use this cell to split inventory in two inventory files:

  - [click here](/edit/notebooks/exercise-04/prod) prod for production servers 
  - [click here](/edit/notebooks/exercise-04/test) test for test servers
  
  

In [23]:
!ansible -i prod -m ping all
!ansible -i test -m ping all

## group_vars
You can move variables out of inventories - eg to simplify inventory scripts - and store them in files under

In [24]:
!mkdir -p group_vars
!touch group_vars/all 
!tree 



.
├── ansible.cfg
├── group_vars
│   ├── all
│   └── example
├── inventory
├── inventory-docker.py
├── prod
├── test
└── untitled.txt

1 directory, 8 files


If you have different inventories, you can store different set of variable in custom files.
The `all` ones will be shared between all inventories

Exercise: 
    
  - [click here to edit](/edit/notebooks/exercise-04/group_vars/all) move all common variables from inventory to all

Inventory variables can store almost everything and even describe the architecture of your deployment


In [25]:
!cat group_vars/example

---
# This is an inventory variable example.

# Basic variables can be strings..
java_version: 1.8.0_131
http_proxy: "http://172.17.0.1:3128"           # ...strings with quotes

# .. numbers
timestamp: 20170501
default_int: 0

# boolean (see http://yaml.org/type/bool.html)
enable_selinux: "yes"
enable_feature: "True"

#
# But you can have
#

a_list:
  - curl
  - wget

a_map:
  name: /etc/hosts
  mode: 0644

a_list_of_map:
  - name: /etc/hosts
    mode: 0644
  - name: /etc/shadow
    mode: 0000


We can even mix and mojo `group_vars` and `inventory`, as we'll see in the next lessons.



